### マニュアル

・付与日のフォルダを作る<br>
・付与日のフォルダの中に、bookフォルダとcheckフォルダを作る<br>
・すべてを実行



In [62]:
import numpy as np
import pandas as pd
import os
import glob
import openpyxl
import xlrd

In [63]:

# 対象のディレクトリパスを指定
date_directory_path = r'C:/Users/hj7724/python/Jupyter_project/ga4entity_check/data/'
# ディレクトリ内のフォルダー名を取得
date_folder_list = [name for name in os.listdir(date_directory_path) if os.path.isdir(os.path.join(date_directory_path, name))]

if date_folder_list:
    grant_date = max(date_folder_list)
    print(f'Grant Date: {grant_date}')

Grant Date: 1127


In [64]:
#権限付与日のフォルダーを命名
#path = os.path.join(date_directory_path,grant_date)
#bookフォルダーを命名
#book_path = os.path.join(path,"book")
#checkフォルダーを命名
#check_path = os.path.join(path,"check")

In [65]:
#新たなデータフレームを作成
columns = ['client','mail', 'propety', 'num_propety', 'name_propety']
all_book = pd.DataFrame(columns=columns)

In [66]:
#付与日フォルダのディレクトリ指定
path = "C:/Users/hj7724/python/Jupyter_project/ga4entity_check/data/{grant_date}".format(grant_date = grant_date)

# bookフォルダのパスを作成
book_path = os.path.join(path, "book")

# bookフォルダ内のExcelファイル名をリストで取得
xlsx_file_list = [os.path.basename(f) for f in glob.glob(os.path.join(book_path, "*.xlsx"))]

# 結果の表示
print(xlsx_file_list)


['20241121_吉田様.xlsx', '20241121_幸田様.xlsx', '20241121_河村様.xlsx', '20241122_吉川様.xlsx', '20241122_清水様.xlsx', '20241122_秋澤様.xlsx', '20241125_中原様.xlsx', '20241125_野田様.xlsx', '20241126_小倉様.xlsx', '20241126_小田様.xlsx', '20241126_平様.xlsx', '20241126_森永様.xlsx', '20241126_田中様.xlsx']


In [67]:
# not_cip_filename を空の集合として初期化
not_cip_filename = []

In [68]:
def make_all_book(file, num, book_path, all_book,not_cip_filename):
    # Excelファイル名を取得
    xlsx_name = file[num]

    # ファイル名から名前を抽出（数字_名前.xlsx の形式を前提）
    client_name = xlsx_name.split('_')[1].replace('.xlsx', '')

    # Excelファイルを開く
    excel_path = os.path.join(book_path, xlsx_name)
    data = pd.read_excel(excel_path, sheet_name="依頼用シート")

    # Pandasデータフレームに変換
    df = pd.DataFrame(data)
    
    # 4行目以降を取得
    df = df[3:]
    
    # F列とH列を取り出す
    df = df.iloc[:, [5, 7]]
    
    # 列名を変更（F列を 'mail' に、H列を 'propety' に変更）
    df.columns = ['mail', 'propety']
    
    # 最下行の'end'を削除
    df = df.dropna(how='all').replace('end', np.nan).dropna(how='all')

    # データフレームを整形
    new_rows = []
    for index, row in df.iterrows():
        name = row['mail']
        items = row['propety'].split('\n')
        for item in items:
            new_rows.append({'mail': name, 'propety': item})

    df = pd.DataFrame(new_rows)
    
    # プロパティ名を分割
    df[['num_propety', 'name_propety']] = df['propety'].str.extract(r'(\d{3,4})_(.+)')
    
    # num_propety列を数値型に変換
    df['num_propety'] = pd.to_numeric(df['num_propety'], errors='coerce')

    # client列を追加し、値を設定（ファイル名から抽出した名前）
    df.insert(0, 'client', client_name)
    
    ########
    # メールアドレスのドメインチェック
    allowed_domain = 'kddi-ga.auone.jp'
    different_domain = df['mail'].apply(lambda x: x.split('@')[1] != allowed_domain)

    if any(different_domain):
        print(f"CIPアカウント以外のアカウントが含まれています。対象ファイルは、{xlsx_name}です。")
        not_cip_filename.append({xlsx_name})
        return [all_book,not_cip_filename]
    else:
        all_book = pd.concat([all_book, df], ignore_index=True)
        return [all_book,not_cip_filename]
    ##############
    
    # 全体のデータフレームに追加
    #all_book = pd.concat([all_book, df], ignore_index=True)
    
    #return all_book

In [69]:
make_all_book(xlsx_file_list, 4, book_path, all_book,not_cip_filename)[0]

,client,mail,propety,num_propety,name_propety
0,清水様,yoshihiro.fujiwara@kddi-ga.auone.jp,520_TELASA,520,TELASA
1,清水様,yoshihiro.fujiwara@kddi-ga.auone.jp,546_TELASA,546,TELASA
2,清水様,yoshihiro.fujiwara@kddi-ga.auone.jp,566_TELASAWebTV,566,TELASAWebTV
3,清水様,mizuka.fuji@kddi-ga.auone.jp,520_TELASA,520,TELASA
4,清水様,mizuka.fuji@kddi-ga.auone.jp,546_TELASA,546,TELASA
5,清水様,mizuka.fuji@kddi-ga.auone.jp,566_TELASAWebTV,566,TELASAWebTV


In [70]:
# 関数を呼び出して処理を実行
# 例えば、num=0 の時に関数を呼び出すとしたら以下のように
#all_book = make_all_book(xlsx_file_list, 0, book_path, all_book)
#for文でmake_all_book関数を回す
# excel_file_list内のすべてのファイルに対してmake_all_book関数を実行

for num in range(len(xlsx_file_list)):
    make_all_book(xlsx_file_list, num, book_path, all_book,not_cip_filename)
    #[all_book,not_cip_filename] = make_all_book(xlsx_file_list, num, book_path, all_book,not_cip_filename)
    #not_cip_filename = make_all_book(xlsx_file_list, num, book_path, all_book,not_cip_filename)[1]

# client列とnum_propety列でソート
all_book = all_book.sort_values(by=['client', 'num_propety']).reset_index(drop=True)

In [71]:
print_not_cip_filename = str(not_cip_filename)[1:-1]
print_not_cip_filename

''

In [72]:
# client列とnum_propety列でソート
all_book = all_book.sort_values(by=['client', 'num_propety']).reset_index(drop=True)

In [73]:
#CIPアカウント以外が含まれる場合アラートを出す
# 特定のドメイン
#allowed_domain = 'kddi-ga.auone.jp'
# フィルタリング条件
#different_domain = all_book['mail'].apply(lambda x: x.split('@')[1] != allowed_domain)
# フィルタリング結果の検出
if len(not_cip_filename) != 0:
    print("CIPアカウント以外のアカウントが含まれています。CIPアカウント以外が含まれている申請書は、{name}です。".format(name = print_not_cip_filename))
else:
    # all_bookをExcelファイルとして保存する
    save_path = os.path.join(path, 'all_book_{grant_date}.xlsx'.format(grant_date = grant_date))
    all_book.to_excel(save_path, index=False)
    # 保存完了メッセージ
    print(f"Excel file saved successfully at {save_path}")


Excel file saved successfully at C:/Users/hj7724/python/Jupyter_project/ga4entity_check/data/1127\all_book_1127.xlsx


In [74]:
#csvファイルが必要なプロパティ一覧
#all_book['propety'].unique()
#all_propety = pd.DataFrame(all_book['propety'].unique())

In [75]:
# all_propetyをExcelファイルとして保存する
#save_path = os.path.join(path, 'all_propety_{grant_date}.xlsx'.format(grant_date = grant_date))
#all_propety.to_excel(save_path, index=False)

# 保存完了メッセージ
#print(f"Excel file saved successfully at {save_path}")